<a href="https://colab.research.google.com/github/MagdaTainy/Arvore_Geradora_Minima-/blob/main/%C3%A1rvore_geradora_m%C3%ADnima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq
import networkx as nx
import matplotlib.pyplot as plt
import random
import os

# ---------- DSU para Kruskal ----------
class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return False
        if self.rank[ra] < self.rank[rb]:
            ra, rb = rb, ra
        self.parent[rb] = ra
        if self.rank[ra] == self.rank[rb]:
            self.rank[ra] += 1
        return True

# ---------- Visualização incremental ----------
def desenhar_passo(G, pos, edges, escolhidas, titulo, delay, pasta, passo):
    plt.clf()
    nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=600, font_weight="bold")
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): w for u, v, w in edges})
    nx.draw_networkx_edges(G, pos, edgelist=escolhidas, width=3, edge_color="red")
    plt.title(titulo)

    # Salvando em arquivo PNG
    nome_arquivo = os.path.join(pasta, f"{titulo.replace(' ', '_')}_passo_{passo}.png")
    plt.savefig(nome_arquivo)

    plt.pause(delay)

# ---------- Kruskal passo a passo ----------
def kruskal_animado(n, edges, delay, pasta):
    edges_sorted = sorted(edges, key=lambda x: x[2])
    dsu = DSU(n)
    mst_edges = []
    total = 0

    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)

    plt.figure(figsize=(6, 5))
    passo = 0
    for u, v, w in edges_sorted:
        if dsu.union(u, v):
            mst_edges.append((u, v, w))
            total += w
        passo += 1
        desenhar_passo(G, pos, edges, [(a, b) for a, b, _ in mst_edges], "Kruskal - Construindo MST", delay, pasta, passo)

    plt.show()
    return total, mst_edges

# ---------- Prim passo a passo ----------
def prim_animado(n, adj, edges, delay, pasta, start=0):
    visited = [False] * n
    min_heap = [(0, start, -1)]
    mst_edges = []
    total = 0

    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)

    plt.figure(figsize=(6, 5))
    passo = 0
    while min_heap and len(mst_edges) < n - 1:
        w, u, parent = heapq.heappop(min_heap)
        if visited[u]:
            continue
        visited[u] = True
        if parent != -1:
            mst_edges.append((parent, u, w))
            total += w
        for v, wt in adj[u]:
            if not visited[v]:
                heapq.heappush(min_heap, (wt, v, u))

        passo += 1
        desenhar_passo(G, pos, edges, [(a, b) for a, b, _ in mst_edges], "Prim - Construindo MST", delay, pasta, passo)

    plt.show()
    return total, mst_edges

# ---------- Gerar grafo aleatório ----------
def gerar_grafo_aleatorio(n, densidade=0.5, peso_min=1, peso_max=20):
    edges = []
    adj = [[] for _ in range(n)]
    max_edges = n * (n - 1) // 2
    num_edges = int(max_edges * densidade)

    possiveis = [(i, j) for i in range(n) for j in range(i + 1, n)]
    escolhidas = random.sample(possiveis, num_edges)

    for u, v in escolhidas:
        w = random.randint(peso_min, peso_max)
        edges.append((u, v, w))
        adj[u].append((v, w))
        adj[v].append((u, w))

    return edges, adj

# ---------- Programa Principal ----------
if __name__ == "__main__":
    print("Árvore Geradora Mínima - Interativa com Animação e Salvamento de Passos")

    pasta = "resultados_mst"
    os.makedirs(pasta, exist_ok=True)

    modo = input("Deseja digitar o grafo ou gerar automaticamente? (manual/auto): ").strip().lower()

    if modo == "manual":
        n = int(input("Digite o número de vértices: "))
        m = int(input("Digite o número de arestas: "))

        edges = []
        adj = [[] for _ in range(n)]

        print("Digite as arestas no formato: u v peso (vértices de 0 até n-1)")
        for _ in range(m):
            u, v, w = input().split()
            u, v, w = int(u), int(v), float(w)
            edges.append((u, v, w))
            adj[u].append((v, w))
            adj[v].append((u, w))

    else:
        n = int(input("Digite o número de vértices: "))
        densidade = float(input("Digite a densidade (0 a 1, ex: 0.5): "))
        edges, adj = gerar_grafo_aleatorio(n, densidade)
        print(f"\n✅ Grafo aleatório gerado com {n} vértices e {len(edges)} arestas.")
        print("Arestas:", edges)

    escolha = input("Qual algoritmo deseja rodar primeiro? (kruskal/prim): ").strip().lower()
    delay = float(input("Digite o tempo de pausa entre passos (ex: 0.5): "))

    if escolha == "kruskal":
        peso_k, mst_k = kruskal_animado(n, edges, delay, pasta)
        print("\nResultado Kruskal:")
        print("Peso total:", peso_k, " Arestas:", mst_k)

        peso_p, mst_p = prim_animado(n, adj, edges, delay, pasta, start=0)
        print("\nResultado Prim:")
        print("Peso total:", peso_p, " Arestas:", mst_p)

    else:
        peso_p, mst_p = prim_animado(n, adj, edges, delay, pasta, start=0)
        print("\nResultado Prim:")
        print("Peso total:", peso_p, " Arestas:", mst_p)

        peso_k, mst_k = kruskal_animado(n, edges, delay, pasta)
        print("\nResultado Kruskal:")
        print("Peso total:", peso_k, " Arestas:", mst_k)

    if peso_k == peso_p:
        print("\n✅ Ambos os algoritmos chegaram ao mesmo peso total da MST.")

    print(f"\n📂 Todas as imagens dos passos foram salvas na pasta: {pasta}/")


Árvore Geradora Mínima - Interativa com Animação e Salvamento de Passos
